In [174]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [175]:
df = pd.read_csv('/kaggle/input/titanic/train.csv')
ne = pd.read_csv('/kaggle/input/titanic/train.csv')

In [176]:
 df['Age'].fillna(df['Age'].mean())

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64

In [177]:
df.drop(columns = ['Name','Cabin','Ticket'],inplace = True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [178]:

df.head()
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

encoder = OneHotEncoder(sparse = False, handle_unknown='ignore')
categorical_col = [col for col in df.columns if df[col].dtype == 'O']
print(categorical_col)


['Sex', 'Embarked']


In [179]:

df['Embarked'] = np.where(df['Embarked'] == '29.69911764705882', 'S', df['Embarked'])
df[categorical_col] = df[categorical_col].astype(str)


In [180]:
new_df = encoder.fit_transform(df[categorical_col])
new_df

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


array([[0., 1., 0., 0., 1., 0.],
       [1., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 1., 0.],
       ...,
       [1., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0.],
       [0., 1., 0., 1., 0., 0.]])

In [181]:
feature_names = encoder.get_feature_names_out()
print(feature_names)
New_df = pd.DataFrame(new_df,columns = feature_names,index = df.index)

['Sex_female' 'Sex_male' 'Embarked_C' 'Embarked_Q' 'Embarked_S'
 'Embarked_nan']


In [182]:
df.drop(columns = ['Sex','Embarked'],inplace = True)
df = pd.concat([df,New_df],axis = 1)

df.head()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,0,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0,0.0
1,2,1,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,3,1,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,0.0
3,4,1,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,0.0
4,5,0,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,0.0


In [183]:

df.isna().sum()
df['Age'] = df['Age'].fillna(df['Age'].mean())
df.head()
y = ne['Survived']
df.drop(columns = ['Survived'],inplace = True)
df.head()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Embarked_nan
0,1,3,22.0,1,0,7.2500,0.0,1.0,0.0,0.0,1.0,0.0
1,2,1,38.0,1,0,71.2833,1.0,0.0,1.0,0.0,0.0,0.0
2,3,3,26.0,0,0,7.9250,1.0,0.0,0.0,0.0,1.0,0.0
3,4,1,35.0,1,0,53.1000,1.0,0.0,0.0,0.0,1.0,0.0
4,5,3,35.0,0,0,8.0500,0.0,1.0,0.0,0.0,1.0,0.0


In [184]:
model = RandomForestClassifier(n_estimators = 100)
y = ne['Survived']
model.fit(df,y)


RandomForestClassifier()

In [185]:
from sklearn.metrics import accuracy_score
y_train = model.predict(df)
print(accuracy_score(y,y_train))



1.0


In [186]:
test_data = pd.read_csv('/kaggle/input/titanic/test.csv')

test_data.drop(columns = ['Name','Cabin','Ticket'],inplace = True)
test_data['Age'].fillna(test_data['Age'].mean())

0      34.50000
1      47.00000
2      62.00000
3      27.00000
4      22.00000
         ...   
413    30.27259
414    39.00000
415    38.50000
416    30.27259
417    30.27259
Name: Age, Length: 418, dtype: float64

In [187]:
test_data.shape

(418, 8)

In [188]:
new_data = encoder.transform(test_data[categorical_col])
feat_names = encoder.get_feature_names_out()
print(feat_names)
new_test_df = pd.DataFrame(new_data,columns = feat_names,index = test_data.index)
new_test_df.head()
test_data = pd.concat([test_data,new_test_df],axis = 1)


['Sex_female' 'Sex_male' 'Embarked_C' 'Embarked_Q' 'Embarked_S'
 'Embarked_nan']


In [190]:
test_data.drop(columns = ['Sex','Embarked'],inplace = True)

In [191]:


#y_pred = model.predict(test_data)

test_data.fillna({'Age': test_data['Age'].mean(), 'Fare': 0},inplace = True)
test_data.head()
test_data.isna().sum()
y_pred = model.predict(test_data)

In [192]:

final_output = test_data['PassengerId']
new_y_pred = pd.DataFrame(y_pred,columns = ['Survived'], index = test_data.index)
final_output = pd.concat([final_output, new_y_pred], axis = 1)


In [193]:
print(y_pred)

[0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 1
 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 1 0 1 0 0 1 0 1 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0
 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 1 1 1 0 0 1 0 1
 0 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0
 1 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1
 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 1
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 1 0
 0 0 1 0 1 0 0 1 0 0 1]


In [195]:
final_output.head()
final_output.to_csv('out.csv',index= False)